In [1]:
# Generated by Selenium IDE
import pytest
import time
import json
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import pandas as pd

os.environ['PATH'] += os.pathsep + 'C:/Users/brokk/geckodriver/'

In [2]:
class ChangeGrades():
  
    def setup_method(self, section, filename):
        self.driver = webdriver.Firefox()
        self.vars = {}
        self.section = section
        self.N = 0
    
        self.N = 0 
        if self.section == 1: 
            self.N = 297
        if self.section == 2: 
            self.N = 289
    
        if self.N == 0: 
            print("You did not enter a valid section number")
            return
    
        if os.path.isfile(filename):
            self.data =  pd.read_csv(filename, index_col=0)
        else: 
            print("FATAL: "+filename+" does not exist!")
        return
        
  
    def teardown_method(self):
        self.driver.quit()
  

    def gradeVal(self, grade):
        
        if grade == 'A': return 2
        if grade == 'A-': return 3
        if grade == 'B': return 4
        if grade == 'B+': return 5
        if grade == 'B-': return 6
        if grade == 'C': return 7
        if grade == 'C+': return 8
        if grade == 'C-': return 9
        if grade == 'D': return 10
        if grade == 'D+': return 11
        if grade == 'F': return 12
        
        return -1

    def changeGrades(self, userid, pwd):
        self.driver.get("https://www.spire.umass.edu/psp/heproda/?cmd=login&languageCd=ENG")
        self.driver.set_window_size(1936, 1096)
        self.driver.find_element(By.ID, "pwd").send_keys(pwd)
        self.driver.find_element(By.ID, "userid").send_keys(userid)
        self.driver.find_element(By.NAME, "Submit").click()
    
        #Goes to the grade change site
        self.driver.get("https://www.spire.umass.edu/psp/heproda/EMPLOYEE/SA/c/UM_SELF_SERVICE.UM_SS_NR_GRADE_CHG.GBL?FolderPath=PORTAL_ROOT_OBJECT.UM_CLASS_MANAGEMENT.UM_SS_NR_GRADE_CHG_GBL&IsFolder=false&IgnoreParamTempl=FolderPath%2cIsFolder")   
    
        #Switch to the frame with the menus
        self.driver.switch_to.frame(0)    
    
        #Select section
        if self.section == 1: 
            self.driver.find_element(By.ID, "UM_SS_NR_WK_UM_SELECT$0").click()
        if self.section == 2: 
            self.driver.find_element(By.ID, "UM_SS_NR_WK_UM_SELECT$1").click()
          
        #Give it time to catch up!
        time.sleep(10)
    
        errorReport = "\n \n" + "STUDENTS **NOT** IN DATAFILE: \n"
    
        #This is where we try to get the grades changed
        for student in range(0, self.N):
    
            #print("student # = ",student)
    
            ID = int(self.driver.find_element(By.ID, "UM_STU_ENRL_VW1_EMPLID$"+str(student)).text)
        
            gradeNum = -1
        
            if ID in self.data.index:         
                grade = self.data.loc[ID].Letter
                gradeNum = self.gradeVal(grade)
                #print("\t",ID,grade,gradeNum)
                if(gradeNum < 0): 
                    print("could not interpret grade = ",grade)
                    continue
        
            else: 
                print("\t",ID," NOT IN DATAFILE!")
                errorReport = errorReport + str(ID) + "\n"
                continue
            
                            
            self.driver.find_element(By.ID, "UM_SS_NR_WK_COURSE_GRADE$42$$"+str(student)).click()
            self.driver.find_element(By.CSS_SELECTOR, "#UM_SS_NR_WK_COURSE_GRADE\\$42\\$\\$"+str(student)+" > option:nth-child("+str(gradeNum)+")").click()
            
            
        for student in range(0, self.N):
            
            ID = int(self.driver.find_element(By.ID, "UM_STU_ENRL_VW1_EMPLID$"+str(student)).text)
            if ID in self.data.index:          
                self.driver.find_element(By.ID, "UM_SS_NR_WK_UM_GC_REASON_SHORT$"+str(student)).click()
                self.driver.find_element(By.CSS_SELECTOR, "#UM_SS_NR_WK_UM_GC_REASON_SHORT\\$"+str(student)+" > option:nth-child(6)").click()
                
            
        print(errorReport)
        

In [3]:
gC = ChangeGrades()

In [4]:
gC.setup_method(section = 2, filename = "grades.csv")